In [1]:
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from scipy.stats import pearsonr
from scipy import spatial

import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import explained_variance_score
from math import sqrt
from sklearn.metrics import r2_score

In [2]:
def pairwise_accuracy(actual, predicted):
    true = 0
    total = 0
    
    cnt = 0
    for i in range(len(actual)):
#         print(cnt, end = "\r")
        for j in range(i+1, len(actual)):
            total += 1

            s1 = actual[i]
            s2 = actual[j]
            b1 = predicted[i]
            b2 = predicted[j]

            result1 = spatial.distance.cosine(s1, s2)
            result2 = spatial.distance.cosine(b1, b2)
            result3 = spatial.distance.cosine(s1, b2)
            result4 = spatial.distance.cosine(s2, b1)

            if(result1 + result2 < result3 + result4):
                true += 1
        cnt += 1

    return(true/total)

In [3]:
def train(X, Y):
    
    splits = 5
    kf = KFold(n_splits=splits)
    
    dataset_X = np.array(X.copy())
    dataset_Y = np.array(Y.copy())
    
    actual = []
    predicted = []

    mean_pa = 0
    
    mean_trainmae = 0
    mean_trainmse = 0
    mean_testmae = 0
    mean_testmse = 0
    
    iterations = 0
        
    for train_index, test_index in kf.split(dataset_X):

        X_train, X_test = dataset_X[train_index], dataset_X[test_index]
        y_train, y_test = dataset_Y[train_index], dataset_Y[test_index]
           
        reg = Ridge(alpha=1.0)
        reg.fit(X_train,y_train)
        
        
        y_trainpred = reg.predict(X_train)
        y_pred=reg.predict(X_test)

        mae = mean_absolute_error(y_train, y_trainpred)
        mse = mean_squared_error(y_train, y_trainpred)
        mean_trainmae += mae
        mean_trainmse += mse
        
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        mean_testmae += mae
        mean_testmse += mse

        actual.extend(y_test)
        predicted.extend(y_pred)
        
        if(iterations == 0):
            mean_pa += pairwise_accuracy(y_test, y_pred)
            iterations +=1
                
    mean_pa/=iterations
    mean_trainmae /= splits
    mean_trainmse /= splits
    mean_testmae /= splits
    mean_testmse /= splits

    return actual,predicted, mean_pa, mean_trainmae, mean_trainmse, mean_testmae, mean_testmse

In [4]:
def train_nosplits(X, Y):

    dataset_X = np.array(X.copy())
    dataset_Y = np.array(Y.copy())
    
    actual = []
    predicted = []

    X_train = dataset_X[:1600]
    X_test = dataset_X[1600:]
    Y_train = dataset_Y[:1600]
    Y_test = dataset_Y[1600:]
        
    reg = Ridge(alpha=1.0)
    reg.fit(X_train,Y_train)

    Y_trainpred = reg.predict(X_train)
    Y_pred=reg.predict(X_test)

    trainmae = mean_absolute_error(Y_train, Y_trainpred)
    trainmse = mean_squared_error(Y_train, Y_trainpred)

    testmae = mean_absolute_error(Y_test, Y_pred)
    testmse = mean_squared_error(Y_test, Y_pred)

    actual.extend(dataset_Y)
    predicted.extend(reg.predict(dataset_X))

    pa = pairwise_accuracy(Y_test, Y_pred)

    return actual,predicted, pa, trainmae, trainmse, testmae, testmse

In [5]:
def train_nn(X, Y):
    
    splits = 5
    kf = KFold(n_splits=splits)
    
    dataset_X = np.array(X.copy())
    dataset_Y = np.array(Y.copy())
    
    actual = []
    predicted = []  
    
    mean_pa = 0
    
    iterations = 0
    
    mean_trainmae = 0
    mean_trainmse = 0
    mean_testmae = 0
    mean_testmse = 0
    
    for train_index, test_index in kf.split(dataset_X):
        
        X_train, X_test = dataset_X[train_index], dataset_X[test_index]
        y_train, y_test = dataset_Y[train_index], dataset_Y[test_index]

        reg = MLPRegressor(hidden_layer_sizes=(256, 256),activation="relu" , max_iter=500).fit(X_train, y_train)

        y_trainpred = reg.predict(X_train)
        y_pred=reg.predict(X_test)
        
        mae = mean_absolute_error(y_train, y_trainpred)
        mse = mean_squared_error(y_train, y_trainpred)
        mean_trainmae += mae
        mean_trainmse += mse
        
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        mean_testmae += mae
        mean_testmse += mse

        actual.extend(y_test)
        predicted.extend(y_pred)
        
        if(iterations == 0):
            mean_pa += pairwise_accuracy(y_test, y_pred)
            iterations +=1
        
    mean_pa/=iterations
    mean_trainmae /= splits
    mean_trainmse /= splits
    mean_testmae /= splits
    mean_testmse /= splits

    return actual, predicted, mean_pa, mean_trainmae, mean_trainmse, mean_testmae, mean_testmse

In [6]:
file = open('../data/BOLD5000/BOLD5000_ROIs/ROIs/stim_lists/CSI01_stim_lists.txt','r')
lines = file.readlines()

COCO_images = []
COCO_ind = []
cnt = 0
for line in lines:
    if(line[0:4] == 'COCO'):
        COCO_images.append(line.split('.')[0])
        COCO_ind.append(cnt)
#     elif(line[0:8] == 'rep_COCO'):
#         COCO_images.append(line.split('.')[0][4:])
#         COCO_ind.append(cnt)
    cnt += 1

In [7]:
print(len(COCO_images), len(COCO_ind))

2000 2000


In [8]:
data = np.load('../data/features/BOLD5000/resnet_feat.npy',allow_pickle=True)

In [9]:
data_dict = dict(enumerate(data.flatten(), 1))[1]
print(list(data_dict.keys())[:5])

['n03721384_567', 'COCO_train2014_000000054001', 'cabin4', 'n07742313_20683', 'n02268853_504']


In [10]:
ROIS = ['LHPPA', 'RHPPA', 'LHLOC', 'RHLOC', 'LHEarlyVis', 'RHEarlyVis', 'LHOPA', 'RHOPA', 'LHRSC', 'RHRSC']
# layers_resnet = ['block1','block2','block3','block4']
layers_resnet = ['block1', 'block3','block4']
TR = ['TR1','TR2','TR3','TR4','TR5','TR34']

In [11]:
fmri_mean = dict()

for roi in ROIS:
    
    fmri_mean[roi] = 0
    
    for tr in TR:
        
        cur_fmri = loadmat("../data/BOLD5000/BOLD5000_ROIs/ROIs/CSI1/mat/CSI1_ROIs_" + tr + ".mat")
        fmri_mean[roi] += cur_fmri[roi]
        
    fmri_mean[roi] /= len(TR)

In [12]:
fmri_mean["LHEarlyVis"].shape

(5254, 210)

## For individual ROIs

In [12]:
predicted_dict = dict()

for layer in layers_resnet:

    predicted_dict[layer] = dict()

    vis_feats = []
    for img in COCO_images:
        vis_feats.append(data_dict[img][layer])
    vis_feats = np.array(vis_feats)

    for roi in ROIS:

        predicted_dict[layer][roi] = dict()

        voxels = fmri_mean[roi][COCO_ind]
        
        print("Ridge", layer, roi)
        actual,predicted,mean_pa,mean_trainmae, mean_trainmse, mean_testmae, mean_testmse = train(voxels, vis_feats)
        mae = mean_absolute_error(actual, predicted)
        mse = mean_squared_error(actual, predicted)
        
        print("PA:", round(mean_pa,3), ", MAE:", round(mae,3), ", MSE:", round(mse,3))
        print("Stats: Train-> MAE:", round(mean_trainmae,3), "MSE:", round(mean_trainmse,3),
                      "Test-> MAE:", round(mean_testmae,3), "MSE:", round(mean_testmse,3))
        
#         print(actual[0][:10])
#         print(predicted[0][:10])

        for i, img in enumerate(COCO_images):
            predicted_dict[layer][roi][img] = dict()
            predicted_dict[layer][roi][img]["Ridge"] = predicted[i]
        
#         print("NN", layer, roi)
#         actual,predicted, mean_pa,mean_trainmae, mean_trainmse, mean_testmae, mean_testmse = train_nn(voxels, vis_feats) 
#         mae = mean_absolute_error(actual, predicted)
#         mse = mean_squared_error(actual, predicted)
        
#         print("PA:", round(mean_pa,3), ", MAE:", round(mae,3), ", MSE:", round(mse,3))
#         print("Stats: Train-> MAE:", round(mean_trainmae,3), "MSE:", round(mean_trainmse,3),
#                       "Test-> MAE:", round(mean_testmae,3), "MSE:", round(mean_testmse,3))
        
#         print(actual[0][:10])
#         print(predicted[0][:10])

#         for i, img in enumerate(COCO_images):
#           predicted_dict[layer][roi][img]["NN"] = predicted[i]

        print()

Ridge block1 LHPPA
PA: 0.499 , MAE: 0.023 , MSE: 0.001
Stats: Train-> MAE: 0.023 MSE: 0.001 Test-> MAE: 0.023 MSE: 0.001

Ridge block1 RHPPA
PA: 0.5 , MAE: 0.023 , MSE: 0.001
Stats: Train-> MAE: 0.023 MSE: 0.001 Test-> MAE: 0.023 MSE: 0.001

Ridge block1 LHLOC
PA: 0.499 , MAE: 0.023 , MSE: 0.001
Stats: Train-> MAE: 0.023 MSE: 0.001 Test-> MAE: 0.023 MSE: 0.001

Ridge block1 RHLOC
PA: 0.499 , MAE: 0.023 , MSE: 0.001
Stats: Train-> MAE: 0.023 MSE: 0.001 Test-> MAE: 0.023 MSE: 0.001

Ridge block1 LHEarlyVis
PA: 0.499 , MAE: 0.023 , MSE: 0.001
Stats: Train-> MAE: 0.023 MSE: 0.001 Test-> MAE: 0.023 MSE: 0.001

Ridge block1 RHEarlyVis
PA: 0.5 , MAE: 0.023 , MSE: 0.001
Stats: Train-> MAE: 0.023 MSE: 0.001 Test-> MAE: 0.023 MSE: 0.001

Ridge block1 LHOPA
PA: 0.499 , MAE: 0.023 , MSE: 0.001
Stats: Train-> MAE: 0.023 MSE: 0.001 Test-> MAE: 0.023 MSE: 0.001

Ridge block1 RHOPA
PA: 0.499 , MAE: 0.023 , MSE: 0.001
Stats: Train-> MAE: 0.023 MSE: 0.001 Test-> MAE: 0.023 MSE: 0.001

Ridge block1 LHRSC

In [23]:
predicted_dict['block3']['LHPPA']['COCO_train2014_000000420713']["Ridge"].shape

(1024,)

In [24]:
import pickle

with open('res_fmri_predictions.pickle', 'wb') as handle:
    pickle.dump(predicted_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
with open('res_fmri_predictions.pickle', 'rb') as handle:
    b = pickle.load(handle)
    
    print(b['block3']['LHPPA']['COCO_train2014_000000420713']["Ridge"].shape)

(1024,)


## For concatenated ROIs

In [13]:
fmri_mean_concat = fmri_mean[ROIS[0]]

for roi in ROIS[1:]:
    
#     print(np.array(fmri_mean_concat).shape)
#     print(np.array(fmri_mean[roi]).shape)
    
    fmri_mean_concat = np.hstack((fmri_mean_concat, np.array(fmri_mean[roi])))
    
print(fmri_mean_concat.shape)

(5254, 1685)


In [14]:
predicted_dict = dict()
dec = 4

for layer in layers_resnet:

    predicted_dict[layer] = dict()

    vis_feats = []
    for img in COCO_images:
        vis_feats.append(data_dict[img][layer])
    vis_feats = np.array(vis_feats)
    
#     print(vis_feats.shape)
    
    predicted_dict[layer] = dict()

    voxels = fmri_mean_concat[COCO_ind]
    
#     print(voxels.shape)
#     print()

    print("Ridge", layer)
    actual,predicted,mean_pa,mean_trainmae, mean_trainmse, mean_testmae, mean_testmse = train_nosplits(voxels, vis_feats)
    mae = mean_absolute_error(actual, predicted)
    mse = mean_squared_error(actual, predicted)

    print("PA:", round(mean_pa,dec), ", MAE:", round(mae,dec), ", MSE:", round(mse,dec))
    print("Stats: Train-> MAE:", round(mean_trainmae,dec), "MSE:", round(mean_trainmse,dec),
                  "Test-> MAE:", round(mean_testmae,dec), "MSE:", round(mean_testmse,dec))

    print(actual[0][:10])
    print(predicted[0][:10])
    print()
    print(actual[100][:10])
    print(predicted[100][:10])
    

    for i, img in enumerate(COCO_images):
        predicted_dict[layer][img] = dict()
        predicted_dict[layer][img]["Ridge"] = predicted[i]

#     print("NN", layer)
#     actual,predicted, mean_pa,mean_trainmae, mean_trainmse, mean_testmae, mean_testmse = train_nn(voxels, vis_feats) 
#     mae = mean_absolute_error(actual, predicted)
#     mse = mean_squared_error(actual, predicted)

#     print("PA:", round(mean_pa,3), ", MAE:", round(mae,3), ", MSE:", round(mse,3))
#     print("Stats: Train-> MAE:", round(mean_trainmae,3), "MSE:", round(mean_trainmse,3),
#                   "Test-> MAE:", round(mean_testmae,3), "MSE:", round(mean_testmse,3))

# #     print(actual[0][:10])
# #     print(predicted[0][:10])

#     for i, img in enumerate(COCO_images):
#         predicted_dict[layer][img]["NN"] = predicted[i]

    print()
    print()

Ridge block1
PA: 0.4977 , MAE: 0.021 , MSE: 0.0009
Stats: Train-> MAE: 0.0207 MSE: 0.0009 Test-> MAE: 0.0222 MSE: 0.001
[0.15132605 0.26220235 0.06936722 0.22458495 0.08439484 0.18557446
 0.28521696 0.2853805  0.22243957 0.2289311 ]
[0.14803523 0.26126467 0.12118849 0.21216967 0.06860797 0.18609902
 0.26002257 0.26389643 0.22080236 0.22296046]

[0.13844694 0.26243487 0.12746781 0.2043317  0.07667113 0.18239176
 0.257242   0.24169062 0.22664496 0.19637322]
[0.13421518 0.25894727 0.13441305 0.20330088 0.06266021 0.18038919
 0.25068647 0.26443434 0.23089794 0.20789742]


Ridge block3
PA: 0.5904 , MAE: 0.0157 , MSE: 0.0005
Stats: Train-> MAE: 0.0155 MSE: 0.0005 Test-> MAE: 0.0165 MSE: 0.0006
[0.08375028 0.00432697 0.02993551 0.02592753 0.06283576 0.00195744
 0.0696753  0.05565235 0.15793903 0.07846506]
[0.11393919 0.014338   0.03330415 0.02067556 0.04710536 0.00223256
 0.05423164 0.04688638 0.05758874 0.02655052]

[0.10663046 0.00631811 0.02473628 0.00502824 0.02259172 0.00201194
 0.048838

In [17]:
predicted_dict['block3']['COCO_train2014_000000420713']

{'Ridge': array([0.11393919, 0.014338  , 0.03330415, ..., 0.02495207, 0.04568773,
        0.0461085 ])}

In [18]:
import pickle

with open('res_fmri__concat_predictions.pickle', 'wb') as handle:
    pickle.dump(predicted_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)